# javascript-extension

## Running Javascript from a notebook in JupyterLab

We can use `IPython.display.Javascript`:

In [1]:
from IPython.display import Javascript

Javascript('console.log("hello");')

<IPython.core.display.Javascript object>

We can also use the `%%javascript` magic provided by ipython:

In [2]:
%%javascript
    console.log('hello')

<IPython.core.display.Javascript object>

## Globals

javascript-extension exposes the folloiwing globals:

* `window`
* `document`
* `element`: The DOM node of the cell's output

We can access properties on `window`:

In [3]:
%%javascript
    console.log(window.location.href)

<IPython.core.display.Javascript object>

We can append elements to the output element:

In [4]:
%%javascript
    element.innerHTML = '<pre>hello</pre>'

<IPython.core.display.Javascript object>

## Using Javascript libraries

We can add d3.js to the _window_:

In [5]:
%%javascript
    var script = document.createElement('script');
    script.type = 'text/javascript';
    script.src = '//cdnjs.cloudflare.com/ajax/libs/d3/4.13.0/d3.js';
    document.head.appendChild(script);
    console.log(window.d3)

<IPython.core.display.Javascript object>

We can use d3.js to append an svg animation to the output:

In [6]:
svg_script = '''
var svg = d3.select(element)
    .append("svg")
    .attr("width", 300)
    .attr("height", 300);    

svg.append("circle")
    .style("stroke", "gray")
    .style("fill", "cyan")
    .attr("r", 130)
    .attr("cx", 150)
    .attr("cy", 150)
    .transition()
        .delay(100)
        .duration(10000)  
        .attr("r", 10)
        .attr("cx", 150)
        .style("fill", "blue"); 
'''

Javascript(svg_script)

<IPython.core.display.Javascript object>

We can use d3.js to visualize data (from a URL) to the output:

In [7]:
d3_script = '''
var parseDate = d3.timeParse("%m/%d/%Y %H:%M:%S %p"),
    formatCount = d3.format(",.0f");

var margin = {top: 10, right: 30, bottom: 30, left: 30},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

var x = d3.scaleTime()
    .domain([new Date(2015, 0, 1), new Date(2016, 0, 1)])
    .rangeRound([0, width]);

var y = d3.scaleLinear()
    .range([height, 0]);

var histogram = d3.histogram()
    .value(function(d) { return d.date; })
    .domain(x.domain())
    .thresholds(x.ticks(d3.timeWeek));

var svg = d3.select(element).append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

svg.append("g")
    .attr("class", "axis axis--x")
    .attr("transform", "translate(0," + height + ")")
    .call(d3.axisBottom(x));

d3.csv("https://gist.githubusercontent.com/mbostock/b2fee5dae98555cf78c9e4c5074b87c3/raw/ef984920d0d607ab06af4b5a325d09cbd6c43bd5/homicides.csv", type, function(error, data) {
  if (error) throw error;

  var bins = histogram(data);

  y.domain([0, d3.max(bins, function(d) { return d.length; })]);

  var bar = svg.selectAll(".bar")
      .data(bins)
    .enter().append("g")
      .attr("class", "bar")
      .attr("transform", function(d) { return "translate(" + x(d.x0) + "," + y(d.length) + ")"; });

  bar.append("rect")
      .attr("x", 1)
      .attr("width", function(d) { return x(d.x1) - x(d.x0) - 1; })
      .attr("height", function(d) { return height - y(d.length); });

  bar.append("text")
      .attr("dy", ".75em")
      .attr("y", 6)
      .attr("x", function(d) { return (x(d.x1) - x(d.x0)) / 2; })
      .attr("text-anchor", "middle")
      .text(function(d) { return formatCount(d.length); });
});

function type(d) {
  d.date = parseDate(d.date);
  return d;
}
'''

Javascript(d3_script)

<IPython.core.display.Javascript object>

Use d3.js to visualize data (from Python) to the output:

In [8]:
import pandas as pd
from string import Template

data = pd.read_csv(
    'https://gist.githubusercontent.com/mbostock/b2fee5dae98555cf78c9e4c5074b87c3/raw/ef984920d0d607ab06af4b5a325d09cbd6c43bd5/homicides.csv', 
    parse_dates=[2],
)

d3_script = Template('''
var parseDate = d3.timeParse("%m/%d/%Y %H:%M:%S %p"),
    formatCount = d3.format(",.0f");

var margin = {top: 10, right: 30, bottom: 30, left: 30},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

var x = d3.scaleTime()
    .domain([new Date(2015, 0, 1), new Date(2016, 0, 1)])
    .rangeRound([0, width]);

var y = d3.scaleLinear()
    .range([height, 0]);

var histogram = d3.histogram()
    .value(function(d) { return d.date; })
    .domain(x.domain())
    .thresholds(x.ticks(d3.timeWeek));

var svg = d3.select(element).append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

svg.append("g")
    .attr("class", "axis axis--x")
    .attr("transform", "translate(0," + height + ")")
    .call(d3.axisBottom(x));
    
var data = $data;

var bins = histogram(data);

y.domain([0, d3.max(bins, function(d) { return d.length; })]);

var bar = svg.selectAll(".bar")
    .data(bins)
    .enter().append("g")
    .attr("class", "bar")
    .attr("transform", function(d) { return "translate(" + x(d.x0) + "," + y(d.length) + ")"; });

bar.append("rect")
    .attr("x", 1)
    .attr("width", function(d) { return x(d.x1) - x(d.x0) - 1; })
    .attr("height", function(d) { return height - y(d.length); });

bar.append("text")
    .attr("dy", ".75em")
    .attr("y", 6)
    .attr("x", function(d) { return (x(d.x1) - x(d.x0)) / 2; })
    .attr("text-anchor", "middle")
    .text(function(d) { return formatCount(d.length); });

function type(d) {
  d.date = parseDate(d.date);
  return d;
}
''')

Javascript(d3_script.substitute(data=data.to_json(orient='records')))

<IPython.core.display.Javascript object>

## Running HTML with inline scripts from a notebook in JupyterLab

We can use `IPython.display.HTML`:

In [9]:
from IPython.display import HTML

HTML('<span>Hello</span>')

In [10]:
HTML('<script src="//cdnjs.cloudflare.com/ajax/libs/d3/4.13.0/d3.js" type="text/javascript" />')

In [11]:
svg_script = '''<div id="d3-circle-animation"></div>
<script type="text/javascript">
var svg = d3.select("#d3-circle-animation")
    .append("svg")
    .attr("width", 300)
    .attr("height", 300);    

svg.append("circle")
    .style("stroke", "gray")
    .style("fill", "cyan")
    .attr("r", 130)
    .attr("cx", 150)
    .attr("cy", 150)
    .transition()
        .delay(100)
        .duration(10000)  
        .attr("r", 10)
        .attr("cx", 150)
        .style("fill", "blue"); 

</script>'''

HTML(svg_script)